In [1]:
import requests
import json
import json
import pandas as pd
from pyspark.sql import SparkSession

In [2]:


# this code scrapes the weather APIs
url = """https://archive-api.open-meteo.com/v1/era5?latitude=21.03&longitude=105.85&start_date=2010-01-01&end_date=2024-11-30&hourly=temperature_2m,precipitation,wind_speed_10m"""
response = requests.get(url)
print(response)
data_json = response.json()
# print(json.dumps(data_json, indent=4))
# # this code structures the retrieved data in a form of DataFrame.
# df_json = {}
# for  item in list(data_json['hourly_units'].keys()) :
#     df_json[item] = data_json['hourly'][item]
# df_pandas = pd.DataFrame(df_json)  
# df_pandas['time'] = pd.to_datetime(df_pandas['time'])
# df_pandas   

<Response [200]>


In [3]:



# 1. Chuyển đổi dữ liệu JSON thành DataFrame pandas
hourly_data = data_json["hourly"]
df = pd.DataFrame(hourly_data)

df



,time,temperature_2m,precipitation,wind_speed_10m
0,2010-01-01T00:00,15.7,0.0,7.3
1,2010-01-01T01:00,15.8,0.0,7.1
2,2010-01-01T02:00,16.3,0.0,6.8
3,2010-01-01T03:00,16.8,0.0,6.6
4,2010-01-01T04:00,17.3,0.1,6.1
...,...,...,...,...
130747,2024-11-30T19:00,16.6,0.0,2.2
130748,2024-11-30T20:00,16.1,0.0,1.9
130749,2024-11-30T21:00,15.8,0.0,3.4
130750,2024-11-30T22:00,15.4,0.0,3.5


In [4]:
import pandas as pd
import numpy as np

# Dữ liệu gốc
df['datetime'] = pd.to_datetime(df['time'])  # Chuyển đổi cột time sang datetime
df.rename(columns={
    'temperature_2m': 'temp_c',
    'precipitation': 'precip_mm',
    'wind_speed_10m': 'wind_mph'
}, inplace=True)

# Thêm các cột cố định
df['name'] = "Hanoi"
df['country'] = "Vietnam"
df['latitude'] = 21.0333
df['longitude'] = 105.85

# Hàm ánh xạ giá trị cho cột condition
def map_condition(row):
    precip = row['precip_mm']
    wind = row['wind_mph']
    temp = row['temp_c']
    
    # Xử lý các trường hợp mưa
    if precip > 10:
        return "Heavy rain"
    elif 5 < precip <= 10:
        return "Moderate rain"
    elif 1 < precip <= 5:
        return "Light rain"
    elif precip > 0:
        return "Patchy light rain"
    
    # Xử lý các trường hợp gió mạnh
    if wind > 20:
        return "Moderate or heavy rain with thunder" if precip > 0 else "Thundery outbreaks possible"
    elif 10 < wind <= 20:
        return "Light rain shower" if precip > 0 else "Windy"
    
    # Xử lý dựa trên nhiệt độ
    if temp < 5:
        return "Fog"
    elif 5 <= temp < 15:
        return "Mist"
    elif temp > 30:
        return "Sunny"
    elif 25 <= temp <= 30:
        return "Partly cloudy"
    
    # Mặc định các điều kiện khác
    return np.random.choice([
        "Cloudy", "Clear", "Overcast", "Patchy light drizzle"
    ])

# Ánh xạ giá trị cho cột condition
df['condition'] = df.apply(map_condition, axis=1)

# Chọn các cột theo đúng schema mới
df_final = df[[
    'datetime', 'name', 'country', 'latitude', 'longitude', 
    'temp_c', 'wind_mph', 'precip_mm', 'condition'
]]

# Kiểm tra kết quả
print(df_final.head())

# Lưu dữ liệu sang file CSV
output_file = "weather_data_transformed.csv"
df_final.to_csv(output_file, index=False)
print(f"Dữ liệu đã được lưu vào file {output_file}")


             datetime   name  country  latitude  longitude  temp_c  wind_mph  \
0 2010-01-01 00:00:00  Hanoi  Vietnam   21.0333     105.85    15.7       7.3   
1 2010-01-01 01:00:00  Hanoi  Vietnam   21.0333     105.85    15.8       7.1   
2 2010-01-01 02:00:00  Hanoi  Vietnam   21.0333     105.85    16.3       6.8   
3 2010-01-01 03:00:00  Hanoi  Vietnam   21.0333     105.85    16.8       6.6   
4 2010-01-01 04:00:00  Hanoi  Vietnam   21.0333     105.85    17.3       6.1   

   precip_mm          condition  
0        0.0              Clear  
1        0.0             Cloudy  
2        0.0           Overcast  
3        0.0             Cloudy  
4        0.1  Patchy light rain  
Dữ liệu đã được lưu vào file weather_data_transformed.csv


In [5]:
output_file = "weather_data_transformed2024.csv"
df_final.to_csv(output_file, index=False)
print(f"Dữ liệu đã được lưu vào file {output_file}")

Dữ liệu đã được lưu vào file weather_data_transformed2024.csv


In [ ]:
# import requests
# import pandas as pd

# # Gửi yêu cầu tới API
# url = """https://archive-api.open-meteo.com/v1/era5?latitude=21.03&longitude=105.85&start_date=2015-01-01&end_date=2023-12-31&hourly=temperature_2m,precipitation,wind_speed_10m"""
# response = requests.get(url)

# # Kiểm tra phản hồi từ API
# if response.status_code == 200:
#     # Chuyển đổi dữ liệu JSON thành DataFrame
#     data_json = response.json()
#     hourly_data = data_json["hourly"]
#     df = pd.DataFrame(hourly_data)
    
#     # Chuyển cột 'time' sang định dạng datetime
#     df['time'] = pd.to_datetime(df['time'])
    
#     # Lưu DataFrame vào file CSV
#     output_file = "weather_data.csv"
#     df.to_csv(output_file, index=False)
    
#     print(f"Dữ liệu đã được lưu vào file {output_file}")
# else:
#     print(f"Lỗi khi gọi API: {response.status_code}")


Dữ liệu đã được lưu vào file weather_data.csv


In [ ]:
# # 2. Khởi tạo SparkSession
# spark = SparkSession.builder \
#     .appName("Weather Data Processing") \
#     .config("spark.jars.packages", "io.delta:delta-core_2.12:2.3.0") \
#     .getOrCreate()


your 131072x1 screen size is bogus. expect trouble
24/12/12 14:23:40 WARN Utils: Your hostname, LAPTOP-DN3PCHGN resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/12 14:23:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/hoan123/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hoan123/.ivy2/cache
The jars for the packages stored in: /home/hoan123/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7ff734e7-6550-4750-8f94-1188c6f976e2;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 248ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |  

In [ ]:
# # 3. Chuyển đổi DataFrame pandas sang DataFrame Spark
# spark_df = spark.createDataFrame(df)

# # 4. Ghi dữ liệu ra Delta Lake
# delta_path = "..delta_lake/delta_lake_table_5"
# spark_df.write.format("delta").mode("overwrite").save(delta_path)

# print("Dữ liệu đã được ghi ra Delta Lake tại:", delta_path)

Py4JJavaError: An error occurred while calling o110.save.
: com.google.common.util.concurrent.ExecutionError: java.lang.NoSuchMethodError: 'java.lang.String org.apache.spark.ErrorInfo.messageFormat()'
	at com.google.common.cache.LocalCache$Segment.get(LocalCache.java:2261)
	at com.google.common.cache.LocalCache.get(LocalCache.java:4000)
	at com.google.common.cache.LocalCache$LocalManualCache.get(LocalCache.java:4789)
	at org.apache.spark.sql.delta.DeltaLog$.getDeltaLogFromCache$1(DeltaLog.scala:808)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:818)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:650)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:163)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:48)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.sideEffectResult$lzycompute(commands.scala:75)
	at org.apache.spark.sql.execution.com

In [ ]:
# spark_df.show()


+----------------+--------------+-------------+--------------+
|            time|temperature_2m|precipitation|wind_speed_10m|
+----------------+--------------+-------------+--------------+
|2015-01-01T00:00|           9.7|          0.0|           5.6|
|2015-01-01T01:00|          12.9|          0.0|           6.5|
|2015-01-01T02:00|          16.9|          0.0|           6.8|
|2015-01-01T03:00|          19.5|          0.0|           8.6|
|2015-01-01T04:00|          21.1|          0.0|           8.9|
|2015-01-01T05:00|          22.1|          0.0|           7.3|
|2015-01-01T06:00|          22.7|          0.0|           7.3|
|2015-01-01T07:00|          22.8|          0.0|           6.8|
|2015-01-01T08:00|          22.8|          0.0|           6.5|
|2015-01-01T09:00|          22.4|          0.0|           6.3|
|2015-01-01T10:00|          21.0|          0.0|           7.1|
|2015-01-01T11:00|          18.5|          0.0|           7.6|
|2015-01-01T12:00|          16.8|          0.0|        